In [66]:
import pandas as pd
import pymysql
from pyhive import hive
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import datetime


In [67]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",50)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [68]:
class hiveParse(object):
    """
    该方法用于读取hive 数据，并转化成pandas dataframe
    """
    def __init__(self, host='', port=10000, user='', passwd='', dbName=''):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName

    def conn_hive(self):
        self.connection = hive.Connection(host=self.host,port=self.port,username=self.user)

    def hive_connect(self,sql_select):
        cur = self.connection.cursor()
        cur.execute(sql_select)
        columns = [col[0] for col in cur.description]
        data = cur.fetchall()
        query_df = pd.DataFrame(data)
        query_df.columns = columns
        return query_df

class mysqldbParse(object):
    """
    该方法用于连接mysql，并执行相应操作
    """

    def __init__(self, host='', port=3306, user='', passwd='', dbName='', charset='utf8', connect_timeout=31536000):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName
        self.charset = charset
        self.connect_timeout = connect_timeout

    def conn_mysql(self):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    password=self.passwd,
                                    db=self.dbName,
                                    charset=self.charset,
                                    connect_timeout=self.connect_timeout,
                                    cursorclass=pymysql.cursors.DictCursor)

    def insert_sql(self, insertSql):
        with self.conn.cursor() as cursor:
            cursor.execute(insertSql)
            self.conn.commit()

    def select_Sql(self, selectSql):
        df = pd.read_sql(selectSql, self.conn)
        return df

    def close(self):
        self.conn.close()

In [69]:
def hive_df(sql):
    host = "58.59.18.61"
    port = 10000
    user = "hdfs"
    dbName = "ods"
    hive_parse = hiveParse(host,port,user,dbName)
    hive_parse.conn_hive()
    hive_pd = hive_parse.hive_connect(sql)
    return hive_pd

def mysql_df(selectSql):
    host = "58.59.11.86"
    port = 3306
    user = "tangdinghai"
    passwd = "Tangdinghai2019!"
    dbName = "reportpublic"
    mysql_parse = mysqldbParse(host,port,user,passwd,dbName)
    mysql_parse.conn_mysql()
    mysql_pd = mysql_parse.select_Sql(selectSql)
    mysql_parse.close()
    return mysql_pd



In [70]:
#### 头条分发模型：有授信额度相关客户级和订单级汇总监控报表---日报

In [71]:
base_dict = {'score_creditcard':['409-560','561-583','584-597','598-608','609-618','619-628','629-638','639-649','650-664','665-752','汇总'],\
             'toutiao_develop':['2.80%','8.09%','10.02%','10.57%','10.63%','11.82%','11.28%','11.31%','11.67%','11.81%','-']}
dairy_report = pd.DataFrame(base_dict)


# dairy_report['product_no'] = "PN00000053"
# dairy_report['product_name'] = '今日头条放心借'

# dairy_report = dairy_report[['product_name','product_no','score_logit','toutiao_develop']]
# dairy_report
                            

In [72]:
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_credit_credit_info") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_credit_credit_info")

table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_credit_credit_apply") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_credit_credit_apply")

table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_loan_invoice") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_loan_invoice")

In [73]:
hive_sql = """


select
	score_creditcard,
	count(distinct(user_id)) as uid_%s,
    count(distinct(session_id)) as sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) >= 409 and cast(t.out_credit_score as int) <= 560 then "409-560"
			when cast(t.out_credit_score as int) >= 561 and cast(t.out_credit_score as int) <= 583 then "561-583"
			when cast(t.out_credit_score as int) >= 584 and cast(t.out_credit_score as int) <= 597 then "584-597"
			when cast(t.out_credit_score as int) >= 598 and cast(t.out_credit_score as int) <= 608 then "598-608"
			when cast(t.out_credit_score as int) >= 609 and cast(t.out_credit_score as int) <= 618 then "609-618"
			when cast(t.out_credit_score as int) >= 619 and cast(t.out_credit_score as int) <= 628 then "619-628"
			when cast(t.out_credit_score as int) >= 629 and cast(t.out_credit_score as int) <= 638 then "629-638"			
			when cast(t.out_credit_score as int) >= 639 and cast(t.out_credit_score as int) <= 649 then "639-649"
			when cast(t.out_credit_score as int) >= 650 and cast(t.out_credit_score as int) <= 664 then "650-664"
			when cast(t.out_credit_score as int) >= 665 and cast(t.out_credit_score as int) <= 752 then "665-752"
		end as score_creditcard
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score,
                    b.OUT_FRONT_RISK_DEGREE as OUT_FRONT_RISK_DEGREE
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

                left JOIN
                (
                    SELECT 
                        session_id,
                        user_id,
                        out_credit_score,
                        OUT_FRONT_RISK_DEGREE
                    FROM ods.ods_toutiao_shouxin_output
                ) AS b ON a.session_id = b.session_id
                

                
        ) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.score_creditcard



"""
# toutiao = spark.sql(hive_sql).toPandas()

In [74]:
def dairly_monitor(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """date_add(CURRENT_DATE,-1)"""
        time2 = """date_add(CURRENT_DATE,-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """date_add(CURRENT_DATE,-%d)"""%daydiff1
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """date_add(add_months(CURRENT_DATE, -%d ),-%d)"""%(daydiff1,daydiff2)
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

def monthly_monitor(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """date_add(last_day(add_months('2019-01-14', -1 )),1)"""
        time2 = """last_day('2019-01-14')"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    else:
        time1 = """date_add(last_day(add_months(CURRENT_DATE, -%d )),1)"""%(monthdiff+1)    #当月第一天：上月最后一天+1天
        time2 = """last_day(add_months(CURRENT_DATE,-%d))"""%monthdiff                  #当月最后一天
        
        date_sql = """select last_day(add_months(CURRENT_DATE,-%d)) as dt"""%monthdiff
        date =  hive_df(date_sql)['dt'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)

    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

In [75]:
dairly_apply_day = dairly_monitor(hive_sql,'today',0,0)
dairly_apply_week = dairly_monitor(hive_sql,'week',8,2)
dairly_apply_month = dairly_monitor(hive_sql,'month',1,2)


join_key = 'score_creditcard'

apply_monitor_dairly = dairy_report.merge(dairly_apply_month,on = join_key ,how='left')\
                                  .merge(dairly_apply_week,on = join_key, how='left')\
                                  .merge(dairly_apply_day,on = join_key, how='left').fillna(0)

col_keep = list(apply_monitor_dairly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_dairly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    apply_monitor_dairly[col][idx] = col_total
    apply_monitor_dairly[col+"_percent"] = apply_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_dairly[col+"_percent"][idx] = col_total 

    
apply_monitor_dairly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_creditcard,toutiao_develop,uid_month,sessionId_month,uid_week,sessionId_week,uid_today,sessionId_today,uid_month_percent,sessionId_month_percent,uid_week_percent,sessionId_week_percent,uid_today_percent,sessionId_today_percent
0,409-560,2.80%,309.0,309.0,242.0,242.0,3.0,3.0,0.235339,0.235160,0.267403,0.267403,0.107143,0.107143
1,561-583,8.09%,323.0,324.0,246.0,246.0,8.0,8.0,0.246002,0.246575,0.271823,0.271823,0.285714,0.285714
2,584-597,10.02%,225.0,225.0,155.0,155.0,3.0,3.0,0.171363,0.171233,0.171271,0.171271,0.107143,0.107143
3,598-608,10.57%,138.0,138.0,95.0,95.0,3.0,3.0,0.105103,0.105023,0.104972,0.104972,0.107143,0.107143
4,609-618,10.63%,80.0,80.0,44.0,44.0,3.0,3.0,0.060929,0.060883,0.048619,0.048619,0.107143,0.107143
5,619-628,11.82%,72.0,72.0,35.0,35.0,1.0,1.0,0.054836,0.054795,0.038674,0.038674,0.035714,0.035714
6,629-638,11.28%,63.0,63.0,32.0,32.0,1.0,1.0,0.047982,0.047945,0.035359,0.035359,0.035714,0.035714
7,639-649,11.31%,27.0,27.0,13.0,13.0,2.0,2.0,0.020564,0.020548,0.014365,0.014365,0.071429,0.071429
8,650-664,11.67%,42.0,42.0,23.0,23.0,2.0,2.0,0.031988,0.031963,0.025414,0.025414,0.071429,0.071429
9,665-752,11.81%,34.0,34.0,20.0,20.0,2.0,2.0,0.025895,0.025875,0.022099,0.022099,0.071429,0.071429


In [76]:
#上线月
apply_month_online = monthly_monitor(hive_sql,'online',-1)
#当月
apply_month_now = monthly_monitor(hive_sql,'not_online',0)
#上月
apply_month_last1 = monthly_monitor(hive_sql,'not_online',1)
#上两月
apply_month_last2 = monthly_monitor(hive_sql,'not_online',2)
#上3月
apply_month_last3 = monthly_monitor(hive_sql,'not_online',3)
#上4月
apply_month_last4 = monthly_monitor(hive_sql,'not_online',4)
#上5月
apply_month_last5 = monthly_monitor(hive_sql,'not_online',5)
#上6月
apply_month_last6 = monthly_monitor(hive_sql,'not_online',6)



join_key = 'score_creditcard'

apply_monitor_monthly = dairy_report.merge(apply_month_online,on = join_key,how='left')\
                                  .merge(apply_month_last6,on = join_key,how='left')\
                                  .merge(apply_month_last5,on = join_key,how='left')\
                                  .merge(apply_month_last4,on = join_key,how='left')\
                                  .merge(apply_month_last3,on = join_key,how='left')\
                                  .merge(apply_month_last2,on = join_key,how='left')\
                                  .merge(apply_month_last1,on = join_key,how='left')\
                                  .merge(apply_month_now,on = join_key,how='left').fillna(0)

col_keep = list(apply_monitor_monthly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_monthly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    apply_monitor_monthly[col][idx] = col_total
    apply_monitor_monthly[col+"_percent"] = apply_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_monthly[col+"_percent"][idx] = col_total 

    
apply_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_creditcard,toutiao_develop,uid_online,sessionId_online,uid_2018_07,sessionId_2018_07,uid_2018_08,sessionId_2018_08,uid_2018_09,sessionId_2018_09,...,uid_2018_09_percent,sessionId_2018_09_percent,uid_2018_10_percent,sessionId_2018_10_percent,uid_2018_11_percent,sessionId_2018_11_percent,uid_2018_12_percent,sessionId_2018_12_percent,uid_2019_01_percent,sessionId_2019_01_percent
0,409-560,2.80%,312.0,312.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.232662,0.232489
1,561-583,8.09%,331.0,332.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.246831,0.247392
2,584-597,10.02%,228.0,228.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.170022,0.169896
3,598-608,10.57%,141.0,141.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105145,0.105067
4,609-618,10.63%,83.0,83.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061894,0.061848
5,619-628,11.82%,73.0,73.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054437,0.054396
6,629-638,11.28%,64.0,64.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047726,0.047690
7,639-649,11.31%,29.0,29.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021626,0.021610
8,650-664,11.67%,44.0,44.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032811,0.032787
9,665-752,11.81%,36.0,36.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026846,0.026826


In [77]:
##### 授信相关日报和月报

In [78]:
shouxin_hive_sql = """


select
	score_creditcard,
	count(distinct(user_id)) as uid_%s,
    count(distinct(session_id)) as sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when cast(t.out_credit_score as int) >= 409 and cast(t.out_credit_score as int) <= 560 then "409-560"
			when cast(t.out_credit_score as int) >= 561 and cast(t.out_credit_score as int) <= 583 then "561-583"
			when cast(t.out_credit_score as int) >= 584 and cast(t.out_credit_score as int) <= 597 then "584-597"
			when cast(t.out_credit_score as int) >= 598 and cast(t.out_credit_score as int) <= 608 then "598-608"
			when cast(t.out_credit_score as int) >= 609 and cast(t.out_credit_score as int) <= 618 then "609-618"
			when cast(t.out_credit_score as int) >= 619 and cast(t.out_credit_score as int) <= 628 then "619-628"
			when cast(t.out_credit_score as int) >= 629 and cast(t.out_credit_score as int) <= 638 then "629-638"			
			when cast(t.out_credit_score as int) >= 639 and cast(t.out_credit_score as int) <= 649 then "639-649"
			when cast(t.out_credit_score as int) >= 650 and cast(t.out_credit_score as int) <= 664 then "650-664"
			when cast(t.out_credit_score as int) >= 665 and cast(t.out_credit_score as int) <= 752 then "665-752"
		end as score_creditcard
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score,
                    b.OUT_FRONT_RISK_DEGREE as OUT_FRONT_RISK_DEGREE
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

                left JOIN
                (
                    SELECT 
                        session_id,
                        user_id,
                        out_credit_score,
                        OUT_FRONT_RISK_DEGREE
                    FROM ods.ods_toutiao_shouxin_output
                ) AS b ON a.session_id = b.session_id
                
                right join 
                (
                
                    select 
                        user_id,
                        APPLY_STATUS
                    from ods_credit_credit_apply
                    where product_no like 'PN00000053%%' and APPLY_STATUS = 12
                
                ) as c on a.user_id = c.user_id
                
        ) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s   
) as a group by a.score_creditcard



"""

# dairly_shouxin_day = dairly_monitor(shouxin_hive_sql,'today',0,0)


In [79]:
dairly_shouxin_day = dairly_monitor(shouxin_hive_sql,'today',0,0)
dairly_shouxin_week = dairly_monitor(shouxin_hive_sql,'week',8,2)
dairly_shouxin_month = dairly_monitor(shouxin_hive_sql,'month',1,2)


join_key = 'score_creditcard'

shouxin_monitor_dairly = dairy_report.merge(dairly_shouxin_month,on = join_key ,how='left')\
                                  .merge(dairly_shouxin_week,on = join_key, how='left')\
                                  .merge(dairly_shouxin_day,on = join_key, how='left').fillna(0)

col_keep = list(shouxin_monitor_dairly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_dairly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    shouxin_monitor_dairly[col][idx] = col_total
    shouxin_monitor_dairly[col+"_percent"] = shouxin_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_dairly[col+"_percent"][idx] = col_total 

    
shouxin_monitor_dairly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_creditcard,toutiao_develop,uid_month,sessionId_month,uid_week,sessionId_week,uid_today,sessionId_today,uid_month_percent,sessionId_month_percent,uid_week_percent,sessionId_week_percent,uid_today_percent,sessionId_today_percent
0,409-560,2.80%,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,561-583,8.09%,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,584-597,10.02%,3.0,3.0,2.0,2.0,0,0,0.021277,0.021277,0.022727,0.022727,0.0,0.0
3,598-608,10.57%,15.0,15.0,14.0,14.0,0,0,0.106383,0.106383,0.159091,0.159091,0.0,0.0
4,609-618,10.63%,21.0,21.0,11.0,11.0,0,0,0.148936,0.148936,0.125000,0.125000,0.0,0.0
5,619-628,11.82%,22.0,22.0,13.0,13.0,0,0,0.156028,0.156028,0.147727,0.147727,0.0,0.0
6,629-638,11.28%,26.0,26.0,17.0,17.0,0,0,0.184397,0.184397,0.193182,0.193182,0.0,0.0
7,639-649,11.31%,10.0,10.0,9.0,9.0,0,0,0.070922,0.070922,0.102273,0.102273,0.0,0.0
8,650-664,11.67%,26.0,26.0,13.0,13.0,0,0,0.184397,0.184397,0.147727,0.147727,0.0,0.0
9,665-752,11.81%,18.0,18.0,9.0,9.0,0,0,0.127660,0.127660,0.102273,0.102273,0.0,0.0


In [80]:
#上线月
shouxin_month_online = monthly_monitor(shouxin_hive_sql,'online',-1)
#当月
shouxin_month_now = monthly_monitor(shouxin_hive_sql,'not_online',0)
#上月
shouxin_month_last1 = monthly_monitor(shouxin_hive_sql,'not_online',1)
#上两月
shouxin_month_last2 = monthly_monitor(shouxin_hive_sql,'not_online',2)
#上3月
shouxin_month_last3 = monthly_monitor(shouxin_hive_sql,'not_online',3)
#上4月
shouxin_month_last4 = monthly_monitor(shouxin_hive_sql,'not_online',4)
#上5月
shouxin_month_last5 = monthly_monitor(shouxin_hive_sql,'not_online',5)
#上6月
shouxin_month_last6 = monthly_monitor(shouxin_hive_sql,'not_online',6)



join_key = 'score_creditcard'

shouxin_monitor_monthly = dairy_report.merge(shouxin_month_online,on = join_key,how='left')\
                                  .merge(shouxin_month_last6,on = join_key,how='left')\
                                  .merge(shouxin_month_last5,on = join_key,how='left')\
                                  .merge(shouxin_month_last4,on = join_key,how='left')\
                                  .merge(shouxin_month_last3,on = join_key,how='left')\
                                  .merge(shouxin_month_last2,on = join_key,how='left')\
                                  .merge(shouxin_month_last1,on = join_key,how='left')\
                                  .merge(shouxin_month_now,on = join_key,how='left').fillna(0)

col_keep = list(shouxin_monitor_monthly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_monthly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    shouxin_monitor_monthly[col][idx] = col_total
    shouxin_monitor_monthly[col+"_percent"] = shouxin_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_monthly[col+"_percent"][idx] = col_total 

    
shouxin_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,score_creditcard,toutiao_develop,uid_online,sessionId_online,uid_2018_07,sessionId_2018_07,uid_2018_08,sessionId_2018_08,uid_2018_09,sessionId_2018_09,...,uid_2018_09_percent,sessionId_2018_09_percent,uid_2018_10_percent,sessionId_2018_10_percent,uid_2018_11_percent,sessionId_2018_11_percent,uid_2018_12_percent,sessionId_2018_12_percent,uid_2019_01_percent,sessionId_2019_01_percent
0,409-560,2.80%,0.0,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,561-583,8.09%,0.0,0.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,584-597,10.02%,3.0,3.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021277,0.021277
3,598-608,10.57%,15.0,15.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.106383,0.106383
4,609-618,10.63%,21.0,21.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.148936,0.148936
5,619-628,11.82%,22.0,22.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.156028,0.156028
6,629-638,11.28%,26.0,26.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.184397,0.184397
7,639-649,11.31%,10.0,10.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070922,0.070922
8,650-664,11.67%,26.0,26.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.184397,0.184397
9,665-752,11.81%,18.0,18.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.127660,0.127660


In [81]:
###########################################
#### 有授信额度相关客户级和订单级汇总监控报表---月报

In [82]:
#### 申请表通过率

In [84]:
accept_rate_dairly = shouxin_monitor_dairly[['score_creditcard']]
idx = len(dairy_report['toutiao_develop']) - 1

for i in range(len(shouxin_monitor_dairly.columns)):
    if(i>=2 and i <=7):
        shouxin_col = shouxin_monitor_dairly.columns[i]
        apply_col = apply_monitor_dairly.columns[i]
#         print(shouxin_col,apply_col)

        accept_col = shouxin_col
        accept_rate_dairly[accept_col+"_rate"] = shouxin_monitor_dairly[shouxin_col]/(apply_monitor_dairly[apply_col])
        accept_rate_dairly[accept_col+"_rate"][idx] = apply_monitor_dairly[apply_col][idx]

accept_rate_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,score_creditcard,uid_month_rate,sessionId_month_rate,uid_week_rate,sessionId_week_rate,uid_today_rate,sessionId_today_rate
0,409-560,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,561-583,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,584-597,0.013333,0.013333,0.012903,0.012903,0.0,0.0
3,598-608,0.108696,0.108696,0.147368,0.147368,0.0,0.0
4,609-618,0.262500,0.262500,0.250000,0.250000,0.0,0.0
5,619-628,0.305556,0.305556,0.371429,0.371429,0.0,0.0
6,629-638,0.412698,0.412698,0.531250,0.531250,0.0,0.0
7,639-649,0.370370,0.370370,0.692308,0.692308,0.0,0.0
8,650-664,0.619048,0.619048,0.565217,0.565217,0.0,0.0
9,665-752,0.529412,0.529412,0.450000,0.450000,0.0,0.0


In [85]:
accept_rate_monthly = shouxin_monitor_monthly[['score_creditcard']]
idx = len(dairy_report['toutiao_develop']) - 1

for i in range(len(shouxin_monitor_monthly.columns)):
    if(i>=2 and i <=16):
        shouxin_col = shouxin_monitor_monthly.columns[i]
        apply_col = apply_monitor_monthly.columns[i]
#       print(shouxin_col,apply_col)

        accept_col = shouxin_col
        accept_rate_monthly[accept_col+"_rate"] = shouxin_monitor_monthly[shouxin_col]/(apply_monitor_monthly[apply_col])
        accept_rate_monthly[accept_col+"_rate"][idx] = apply_monitor_monthly[apply_col][idx]

accept_rate_monthly = accept_rate_monthly.fillna("0")
accept_rate_monthly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,score_creditcard,uid_online_rate,sessionId_online_rate,uid_2018_07_rate,sessionId_2018_07_rate,uid_2018_08_rate,sessionId_2018_08_rate,uid_2018_09_rate,sessionId_2018_09_rate,uid_2018_10_rate,sessionId_2018_10_rate,uid_2018_11_rate,sessionId_2018_11_rate,uid_2018_12_rate,sessionId_2018_12_rate,uid_2019_01_rate
0,409-560,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
1,561-583,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
2,584-597,0.013158,0.013158,0,0,0,0,0,0,0,0,0,0,0,0,0.013158
3,598-608,0.106383,0.106383,0,0,0,0,0,0,0,0,0,0,0,0,0.106383
4,609-618,0.253012,0.253012,0,0,0,0,0,0,0,0,0,0,0,0,0.253012
5,619-628,0.301370,0.301370,0,0,0,0,0,0,0,0,0,0,0,0,0.301370
6,629-638,0.406250,0.406250,0,0,0,0,0,0,0,0,0,0,0,0,0.406250
7,639-649,0.344828,0.344828,0,0,0,0,0,0,0,0,0,0,0,0,0.344828
8,650-664,0.590909,0.590909,0,0,0,0,0,0,0,0,0,0,0,0,0.590909
9,665-752,0.500000,0.500000,0,0,0,0,0,0,0,0,0,0,0,0,0.500000


In [86]:
#### 数据导入hive

In [95]:
base_dict = {'score_creditcard':['409-560','561-583','584-597','598-608','609-618','619-628','629-638','639-649','650-664','665-752','汇总'],\
             'toutiao_develop':['2.80%','8.09%','10.02%','10.57%','10.63%','11.82%','11.28%','11.31%','11.67%','11.81%','-']}
dairy_report = pd.DataFrame(base_dict)

dairy_report['product_no'] = "PN00000053"
dairy_report['product_name'] = '今日头条放心借'
dairy_report['product_state'] = '申请'
dairy_report['statisti_caliber'] = '客户'
dairy_report['report_dim'] = 'score_card'

dairy_report2 = dairy_report[['product_name','product_no','report_dim','product_state','statisti_caliber','score_creditcard','toutiao_develop']]


dairly_customer_app = ['score_creditcard','uid_month_percent','uid_week_percent','uid_today_percent']
dairly_table = apply_monitor_dairly[dairly_customer_app]
 
dairly_customer_app_table = dairy_report2.merge(dairly_table,on='score_creditcard',how = 'left')
dairly_customer_app_table


,product_name,product_no,report_dim,product_state,statisti_caliber,score_creditcard,toutiao_develop,uid_month_percent,uid_week_percent,uid_today_percent
0,今日头条放心借,PN00000053,score_card,申请,客户,409-560,2.80%,0.235339,0.267403,0.107143
1,今日头条放心借,PN00000053,score_card,申请,客户,561-583,8.09%,0.246002,0.271823,0.285714
2,今日头条放心借,PN00000053,score_card,申请,客户,584-597,10.02%,0.171363,0.171271,0.107143
3,今日头条放心借,PN00000053,score_card,申请,客户,598-608,10.57%,0.105103,0.104972,0.107143
4,今日头条放心借,PN00000053,score_card,申请,客户,609-618,10.63%,0.060929,0.048619,0.107143
5,今日头条放心借,PN00000053,score_card,申请,客户,619-628,11.82%,0.054836,0.038674,0.035714
6,今日头条放心借,PN00000053,score_card,申请,客户,629-638,11.28%,0.047982,0.035359,0.035714
7,今日头条放心借,PN00000053,score_card,申请,客户,639-649,11.31%,0.020564,0.014365,0.071429
8,今日头条放心借,PN00000053,score_card,申请,客户,650-664,11.67%,0.031988,0.025414,0.071429
9,今日头条放心借,PN00000053,score_card,申请,客户,665-752,11.81%,0.025895,0.022099,0.071429


In [96]:

for col in apply_monitor_dairly.columns:
    if col == 'score_creditcard' or col.find('uid_'):
        

Index(['score_creditcard', 'toutiao_develop', 'uid_month', 'sessionId_month',
       'uid_week', 'sessionId_week', 'uid_today', 'sessionId_today',
       'uid_month_percent', 'sessionId_month_percent', 'uid_week_percent',
       'sessionId_week_percent', 'uid_today_percent',
       'sessionId_today_percent'],
      dtype='object')

In [88]:
apply_monitor_dairly
shouxin_monitor_dairly
apply_monitor_monthly
shouxin_monitor_monthly
accept_rate_dairly
accept_rate_monthly

,score_creditcard,uid_online_rate,sessionId_online_rate,uid_2018_07_rate,sessionId_2018_07_rate,uid_2018_08_rate,sessionId_2018_08_rate,uid_2018_09_rate,sessionId_2018_09_rate,uid_2018_10_rate,sessionId_2018_10_rate,uid_2018_11_rate,sessionId_2018_11_rate,uid_2018_12_rate,sessionId_2018_12_rate,uid_2019_01_rate
0,409-560,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
1,561-583,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
2,584-597,0.013158,0.013158,0,0,0,0,0,0,0,0,0,0,0,0,0.013158
3,598-608,0.106383,0.106383,0,0,0,0,0,0,0,0,0,0,0,0,0.106383
4,609-618,0.253012,0.253012,0,0,0,0,0,0,0,0,0,0,0,0,0.253012
5,619-628,0.301370,0.301370,0,0,0,0,0,0,0,0,0,0,0,0,0.301370
6,629-638,0.406250,0.406250,0,0,0,0,0,0,0,0,0,0,0,0,0.406250
7,639-649,0.344828,0.344828,0,0,0,0,0,0,0,0,0,0,0,0,0.344828
8,650-664,0.590909,0.590909,0,0,0,0,0,0,0,0,0,0,0,0,0.590909
9,665-752,0.500000,0.500000,0,0,0,0,0,0,0,0,0,0,0,0,0.500000


In [89]:
apply_monitor_dairly

,score_creditcard,toutiao_develop,uid_month,sessionId_month,uid_week,sessionId_week,uid_today,sessionId_today,uid_month_percent,sessionId_month_percent,uid_week_percent,sessionId_week_percent,uid_today_percent,sessionId_today_percent
0,409-560,2.80%,309.0,309.0,242.0,242.0,3.0,3.0,0.235339,0.235160,0.267403,0.267403,0.107143,0.107143
1,561-583,8.09%,323.0,324.0,246.0,246.0,8.0,8.0,0.246002,0.246575,0.271823,0.271823,0.285714,0.285714
2,584-597,10.02%,225.0,225.0,155.0,155.0,3.0,3.0,0.171363,0.171233,0.171271,0.171271,0.107143,0.107143
3,598-608,10.57%,138.0,138.0,95.0,95.0,3.0,3.0,0.105103,0.105023,0.104972,0.104972,0.107143,0.107143
4,609-618,10.63%,80.0,80.0,44.0,44.0,3.0,3.0,0.060929,0.060883,0.048619,0.048619,0.107143,0.107143
5,619-628,11.82%,72.0,72.0,35.0,35.0,1.0,1.0,0.054836,0.054795,0.038674,0.038674,0.035714,0.035714
6,629-638,11.28%,63.0,63.0,32.0,32.0,1.0,1.0,0.047982,0.047945,0.035359,0.035359,0.035714,0.035714
7,639-649,11.31%,27.0,27.0,13.0,13.0,2.0,2.0,0.020564,0.020548,0.014365,0.014365,0.071429,0.071429
8,650-664,11.67%,42.0,42.0,23.0,23.0,2.0,2.0,0.031988,0.031963,0.025414,0.025414,0.071429,0.071429
9,665-752,11.81%,34.0,34.0,20.0,20.0,2.0,2.0,0.025895,0.025875,0.022099,0.022099,0.071429,0.071429


In [65]:
shouxin_monitor_dairly

,score_creditcard,toutiao_develop,apply_uid_month,apply_sessionId_month,apply_uid_week,apply_sessionId_week,apply_uid_today,apply_sessionId_today,apply_uid_month_percent,apply_sessionId_month_percent,apply_uid_week_percent,apply_sessionId_week_percent,apply_uid_today_percent,apply_sessionId_today_percent
0,409-560,2.80%,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,561-583,8.09%,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
2,584-597,10.02%,3.0,3.0,2.0,2.0,0,0,0.021277,0.021277,0.022727,0.022727,0.0,0.0
3,598-608,10.57%,15.0,15.0,14.0,14.0,0,0,0.106383,0.106383,0.159091,0.159091,0.0,0.0
4,609-618,10.63%,21.0,21.0,11.0,11.0,0,0,0.148936,0.148936,0.125000,0.125000,0.0,0.0
5,619-628,11.82%,22.0,22.0,13.0,13.0,0,0,0.156028,0.156028,0.147727,0.147727,0.0,0.0
6,629-638,11.28%,26.0,26.0,17.0,17.0,0,0,0.184397,0.184397,0.193182,0.193182,0.0,0.0
7,639-649,11.31%,10.0,10.0,9.0,9.0,0,0,0.070922,0.070922,0.102273,0.102273,0.0,0.0
8,650-664,11.67%,26.0,26.0,13.0,13.0,0,0,0.184397,0.184397,0.147727,0.147727,0.0,0.0
9,665-752,11.81%,18.0,18.0,9.0,9.0,0,0,0.127660,0.127660,0.102273,0.102273,0.0,0.0
